In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        break

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Import Libraries

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics import classification_report , confusion_matrix , accuracy_score
from sklearn.model_selection import train_test_split

import cv2
#from google.colab.patches import cv2_imshow
from PIL import Image 
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Input, Dense,Conv2D , MaxPooling2D, Flatten,BatchNormalization,Dropout
from tensorflow.keras.preprocessing import image_dataset_from_directory
import tensorflow_hub as hub 

In [ ]:
folder_path_no = "/kaggle/input/brain-mri-images-for-brain-tumor-detection/no"
folder_path_yes = "/kaggle/input/brain-mri-images-for-brain-tumor-detection/yes"

folder_no = os.listdir(folder_path_no)
folder_yes = os.listdir(folder_path_yes)

print(len(folder_no))
print(len(folder_yes))

In [ ]:
folder_no[:4]

## Target Labels

In [ ]:
## We can consider 
   # No Tumor  >> 0
   # Yes Tumor >> 1 

no_label = [0]*len(folder_no)
yes_label = [1]*len(folder_yes)

labels = no_label + yes_label
print(len(labels))

In [ ]:
print(labels[:5])
print(labels[-5:])

## Image data preprocessing 

In [ ]:
# Convert image into numpy array
# Resize image ino 224 x 224 x 3
# Image Visualization
# Scale the image array

In [ ]:
data = []

for img in folder_no:
    image = Image.open("/kaggle/input/brain-mri-images-for-brain-tumor-detection/no/"+img)
    image = image.resize((224,224))
    image = image.convert("RGB")
    image = np.array(image)
    data.append(image)

for img in folder_yes:
    image = Image.open("/kaggle/input/brain-mri-images-for-brain-tumor-detection/yes/"+img)
    image = image.resize((224,224))
    image = image.convert("RGB")
    image = np.array(image)
    data.append(image)

In [ ]:
len(data)

In [ ]:
data[0].shape

In [ ]:
x = np.array(data)
y = np.array(labels)

In [ ]:
x[0]

In [ ]:
y

## Split The data 

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.10,shuffle=True)
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
x_train[0]

In [ ]:
y_train

## Image Data Visualization

In [ ]:
class_labels=["No Tumor","Tumor"]
plt.figure(figsize=(16,20))

for i in range(16):
    plt.subplot(4,4,i+1)
    plt.imshow(x_train[i])
    plt.title(f"{class_labels[y_train[i]]}")
    plt.axis("off")

In [ ]:
nan

## Scale The Data

In [ ]:
x_train_scaled = x_train/255
x_test_scaled = x_test/255

## Create CNN Model

In [ ]:
model = Sequential()
model.add(Input(shape=(224,224,3)))
model.add(Conv2D(filters=80,kernel_size=(3,3),padding="valid", strides=(1,1) ,activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(filters=64,kernel_size=(3,3),padding="valid", strides=(1,1), activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

#model.add(Conv2D(filters=34,kernel_size=(3,3),padding="valid", strides=(1,1), activation="relu"))
#model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())

model.add(Dense(units=500, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(units=500, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(units=1, activation='sigmoid'))


model.compile(optimizer="adam", loss="binary_crossentropy",
             metrics=["accuracy"])


model.summary()

In [ ]:
history = model.fit(x_train_scaled, y_train, epochs=10,
                   validation_data=(x_test_scaled,y_test))  # batch_size=32

## Model Evaluation on test Data

In [ ]:
loss, acc = model.evaluate(x_test_scaled,y_test)
print("Accuracy on Test Data:",acc)

In [ ]:
y_pred = model.predict(x_test_scaled)
y_pred = [1 if i>=0.5 else 0 for i in y_pred]

y_pred[:6]

In [ ]:
y_test[:10]   # Actual Label

In [ ]:
y_pred[:10]  # Predicted Label

## Evaluation Matrix

In [ ]:
print("Confusion Matrix:\n",confusion_matrix(y_test,y_pred))
print()
print("Classification Report:\n",classification_report(y_test,y_pred))

## Actual & Predicted Image 

In [ ]:
class_labels=["No Tumor","Tumor"]
plt.figure(figsize=(16,20))

for i in range(16):
    plt.subplot(4,4,i+1)
    plt.imshow(x_test[i])
    plt.title(f"Actual label:{class_labels[y_test[i]]}\nPredicted label:{class_labels[y_pred[i]]}")
    plt.axis("off")

In [ ]:
nan

## Using Pretrained Model

In [ ]:
path = "https://tfhub.dev/google/efficientnet/b0/classification/1" # 224x224x3
efficient_model = hub.KerasLayer(path,input_shape=(224,224,3),trainable=False)


eff_model = Sequential()
eff_model.add(efficient_model)
eff_model.add(Dense(units=1, activation="sigmoid"))

eff_model.summary()

In [ ]:
eff_model.compile(optimizer="adam",
              loss="binary_crossentropy",
              metrics=["accuracy"])
history_4 = eff_model.fit(x_train_scaled , y_train ,
           epochs=10, validation_data=(x_test_scaled,y_test))

In [ ]:
loss, acc = eff_model.evaluate(x_test_scaled,y_test)
print("Accuracy on Test Data:",acc)

In [ ]:
y_pred = eff_model.predict(x_test_scaled)
y_pred = [1 if i>=0.5 else 0 for i in y_pred]

In [ ]:
print("Confusion Matrix:\n",confusion_matrix(y_test,y_pred))
print()
print("Classification Report:\n",classification_report(y_test,y_pred))

## Actual & Predicted Image on EfficientNet Model

In [ ]:
class_labels=["No Tumor","Tumor"]
plt.figure(figsize=(16,20))

for i in range(16):
    plt.subplot(4,4,i+1)
    plt.imshow(x_test[i])
    plt.title(f"Actual label:{class_labels[y_test[i]]}\nPredicted label:{class_labels[y_pred[i]]}")
    plt.axis("off")

In [ ]:
nan